In [12]:
from bs4 import BeautifulSoup
from operator import add
import re
import codecs
import csv
import numpy as numpy

Bodys = []
terms = []

for file in os.listdir("data/"):
    if file.endswith(".sgm"):
        filename = os.path.join("data", file)
        if ( filename == "data/reut2-003.sgm"):
            with codecs.open(filename, "r", encoding='utf-8', errors='ignore') as fn:
                soup = BeautifulSoup(fn.read(), "html.parser")
                bodies = soup.findAll('body')

                for body in bodies:
                    Bodys.append(str(body))
                    temp = re.sub("(<body>|</body>)", " ", str(body))
                    fixBodys = re.findall("[a-zA-Z]+", temp)

                    for fixBody in fixBodys:
                        terms.append(fixBody.lower())

rowData_term = sc.parallelize(terms)
rowData_term_rdd = rowData_term.map(lambda x : (x, 1)).reduceByKey(add).map(lambda x: x[0]).collect()
print("Map term finished!")

Matrix = []
for term in rowData_term_rdd:
    temp = []
    for Body in Bodys:
        temp.append(Body.count(term))
    Matrix.append(temp)
print("Matrix finished!")

tfMatrix = sc.parallelize(Matrix)
with open ("hw4_1.txt", 'w', newline='') as output:
    writer = csv.writer(output)
    writer.writerows(tfMatrix.collect())
print("Output finished!")

Map term finished!
Matrix finished!
Output finished!


In [14]:
from pyspark.mllib.linalg.distributed import *

tempList = []
i = 0
while i < len(Bodys) :
    if i%2 == 0:
        tempList.append(0)
    else:
        tempList.append(1)
    i+=1
matMaster = RowMatrix(tfMatrix)
dm = Matrices.dense(len(Bodys), 1, tempList)
temp = matMaster.multiply(dm)

with open( "hw4_2.txt", 'w', newline='') as output:
    writer = csv.writer(output)
    writer.writerows(temp.rows.collect())
print("Output finished!")

Output finished!
